In [3]:
import dask.array as da
import dask
import zarr
import cv2
import numpy as np
import matplotlib

In [5]:
from MotionEnergyAnalyzer import MotionEnergyAnalyzer

In [2]:
!pip install matplotlib


     |████████████████████████████████| 9.2 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 91.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 77.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 68.5 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 75.8 MB/s eta 0:00:01


In [4]:
# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import utils
zarr_paths = utils.get_zarr_paths()

Searching for zarrs in /root/capsule/data: 100%|██████████| 1/1 [00:00<00:00, 15252.01it/s]
Searching for zarrs in /root/capsule/data/test_data: 100%|██████████| 1/1 [00:00<00:00, 5405.03it/s]


Found zarr file: /root/capsule/data/test_data/processed_frames_zarr


Searching for zarrs in /root/capsule/data/test_data/processed_frames_zarr: 0it [00:00, ?it/s]


In [14]:
zarr_store_frames = zarr.DirectoryStore(zarr_paths[0])
grayscale_frames = da.from_zarr(zarr_store_frames)

In [15]:
grayscale_frames

dask.array<from-zarr, shape=(228152, 492, 658), dtype=float32, chunksize=(100, 492, 658), chunktype=numpy.ndarray>

In [23]:
crop_x_start, crop_y_start, crop_x_end, crop_y_end = (100, 200, 400, 300)

In [24]:
grayscale_frames

dask.array<from-zarr, shape=(228152, 492, 658), dtype=float32, chunksize=(100, 492, 658), chunktype=numpy.ndarray>

In [26]:
grayscale_frames[:, crop_y_start:crop_y_end, crop_x_start:crop_x_end]

dask.array<getitem, shape=(228152, 100, 300), dtype=float32, chunksize=(100, 100, 300), chunktype=numpy.ndarray>

In [ ]:

# Load the grayscale frames from the Zarr file
zarr_store_frames = zarr.DirectoryStore('video_frames_grayscale.zarr')
grayscale_frames = da.from_zarr(zarr_store_frames, component='frames')

# Compute motion energy as the absolute difference between consecutive frames
# This will be a Dask array operation that only loads and computes chunks as needed
motion_energy = da.abs(grayscale_frames[1:] - grayscale_frames[:-1])

# Define the chunking for efficient memory usage; rechunk if necessary
motion_energy = motion_energy.rechunk((100, 658, 492))  # Adjust based on available memory

# Save motion energy as a new Zarr file
zarr_store_motion = zarr.DirectoryStore('motion_energy_grayscale.zarr')
motion_energy.to_zarr(zarr_store_motion, overwrite=True)

# Compute the sum of motion energy for each frame and save as a NumPy array
# This sums across height and width (658, 492) to get the sum per frame (along the time dimension)
motion_energy_sum = motion_energy.sum(axis=(1, 2)).compute()

# Save the result as a NumPy array
np.save('motion_energy_sum.npy', motion_energy_sum)